[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/mengyulin/CoastalEngineering/blob/master/Showcase/Chap_3/2_DispersionEq.ipynb)

## 載入 Python 函式庫

In [2]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt

# 頻散關係式之波長疊代

\begin{equation}
L = \frac{gT^2}{2\pi}\tanh\frac{2\pi h}{L}
\end{equation}

先猜測波長的初始猜值 $L_0$ 代入等號右側，可以得到猜值 1 $L_1$:

\begin{equation}
L_1 = \frac{gT^2}{2\pi}\tanh\frac{2\pi h}{L_0}
\end{equation}

再將 $L_1$ 代入右側，得到猜值 2。如此依序疊代直到收斂。

## 範例

水波週期 $T = 10~\mathrm{s}$，求出水深 $h = 10~\mathrm{m}$ 處之波長。

設初始猜值 $L_0$ 為深水波長 $L_0 = gT^2/2\pi$ 進行疊代。

In [2]:
T = 10
h = 10
g = 9.81
L = g*T**2/(2*np.pi)
print(f"In deep water, L0 = {L:.4f}")
print()

for i in range(50):
    L = g*T**2/(2*np.pi) * np.tanh(2*np.pi*h/L)
    print(f"i = {i}, L = {L:.4f}")

In deep water, L0 = 156.1310

i = 0, L = 59.6462
i = 1, L = 122.2706
i = 2, L = 73.8436
i = 3, L = 107.9689
i = 4, L = 81.8247
i = 5, L = 100.8134
i = 6, L = 86.4007
i = 7, L = 97.0133
i = 8, L = 89.0094
i = 9, L = 94.9433
i = 10, L = 90.4855
i = 11, L = 93.8023
i = 12, L = 91.3163
i = 13, L = 93.1695
i = 14, L = 91.7824
i = 15, L = 92.8175
i = 16, L = 92.0433
i = 17, L = 92.6214
i = 18, L = 92.1892
i = 19, L = 92.5120
i = 20, L = 92.2707
i = 21, L = 92.4510
i = 22, L = 92.3163
i = 23, L = 92.4169
i = 24, L = 92.3417
i = 25, L = 92.3979
i = 26, L = 92.3559
i = 27, L = 92.3873
i = 28, L = 92.3638
i = 29, L = 92.3814
i = 30, L = 92.3683
i = 31, L = 92.3781
i = 32, L = 92.3707
i = 33, L = 92.3762
i = 34, L = 92.3721
i = 35, L = 92.3752
i = 36, L = 92.3729
i = 37, L = 92.3746
i = 38, L = 92.3733
i = 39, L = 92.3743
i = 40, L = 92.3736
i = 41, L = 92.3741
i = 42, L = 92.3737
i = 43, L = 92.3740
i = 44, L = 92.3738
i = 45, L = 92.3739
i = 46, L = 92.3738
i = 47, L = 92.3739
i = 48, L = 92.37

# 頻散關係式之近似解

## 1. Eckart (1951) 提出之近似解

\begin{equation}
k = \frac{\omega^2}{g\sqrt{\tanh\left(\frac{\omega^2}{g}h \right)}}
\end{equation}

### 測試例
T = 10 s, h = 10 m, 精確解為 L = 92.373 m。上式之近似解為

In [4]:
# Wave condition and parameters
g = 9.81
T = 10
h = 10
omega = 2*np.pi/T

# Approximation solution by Eckart (1951) 
k = omega**2 / (g*np.sqrt(np.tanh(omega**2/g*h)))
L = 2*np.pi/k
print(f"L = {L:.4f} m")

# 誤差百分比 (精確解 L = 92.373)
err = np.abs(L-92.373)/92.373 * 100
print(f"Relative error of L = {err:.2f}%")

L = 96.5019 m
Relative error of L = 4.47%


## 1.1 使用 Newton-Raphson 法提升精確度

由

\begin{equation}
\frac{L}{L_0} = \tanh kh
\end{equation}

可知，

\begin{equation}
k_0h = kh\tanh kh
\end{equation}

定義一函數為

\begin{equation}
f(x) = x\tanh x - k_0h
\end{equation}

則此函數之根即為 $kh$ 之精確解。

Newton-Raphson 法可簡述為 (https://en.wikipedia.org/wiki/Newton%27s_method)

\begin{equation}
kh \simeq x_0 - f(x_0)/f'(x_0)
\end{equation}

其中 $x_0$ 為 $kh$ 之初始猜值，$f'(x)$ 為 $f(x)$ 之一階導數。如此可得 $kh$ 之近似解為

\begin{equation}
kh\simeq x_0\left[\frac{k_0h+(x_0/\cosh x_0)^2}{x_0\tanh x_0 + (x_0/\cosh x_0)^2} \right]
\end{equation}

以下程式為使用 Eckart (1951) 之解作為初始猜值，再使用 Newton-Raphson 法進行一次疊代。

In [4]:
# Wave condition and parameters
g = 9.81
T = 10
h = 10
omega = 2*np.pi/T

# Deep water wave number
k0 = 4*np.pi**2/(g*T**2)  

# Initial guess using approximation solution by Eckart (1951)
k = omega**2 / (g*np.sqrt(np.tanh(omega**2/g*h)))
x0 = k*h

# Iteration using Newton-Raphson method
kh = x0 * (k0*h + (x0/np.cosh(x0))**2)/(x0*np.tanh(x0)+(x0/np.cosh(x0))**2)
L = 2*np.pi*h/kh
print(f"L = {L:.4f} m")

# 誤差百分比 (精確解 L = 92.373)
err = np.abs(L-92.373)/92.373 * 100
print(f"Relative error of L = {err:.2f}%")

L = 92.3272 m
Relative error of L = 0.05%


### Note: 在 Newton-Raphson 法中使用 $k_0h$ 作為初始猜值

* 以下程式為令 $x_0 = k_0h$ 作為初始猜值，以 Newton-Raphon 法進行一次疊代。
* 結果發現誤差達 8.71%，比 Eckart (1951) 之解還差。
* 代表若初始猜值距離精確解較遠時，需要多次疊代方可收斂。

In [5]:
# Wave condition and parameters
g = 9.81
T = 10
h = 10
omega = 2*np.pi/T

# Deep water wave number
k0 = 4*np.pi**2/(g*T**2)  
x0 = k0*h

# Iteration using Newton-Raphson method
kh = x0 * (k0*h + (x0/np.cosh(x0))**2)/(x0*np.tanh(x0)+(x0/np.cosh(x0))**2)
L = 2*np.pi*h/kh
print(f"L = {L:.4f} m")

# 誤差百分比 (精確解 L = 92.373)
err = np.abs(L-92.373)/92.373 * 100
print(f"Relative error of L = {err:.2f}%")

L = 84.3256 m
Relative error of L = 8.71%


## 2. Fenton & McKee (1990) 提出之近似解

\begin{equation}
L = \frac{gT^2}{2\pi}\left[\tanh\left(\frac{\omega^2h}{g}\right)^{3/4}\right]^{2/3}
\end{equation}

Source: Fenton, J. D. & McKee, W. D. (1990) On calculating the lengths of water waves, Coastal Engineering 14, 499–513.

### 測試例
T = 10 s, h = 10 m, 精確解為 L = 92.373 m。上式之近似解為

In [5]:
# Wave condition and parameters
g = 9.81
T = 10
h = 10
omega = 2*np.pi/T

# Approximation solution by Eckart (1951) 
L = g*T**2/(2*np.pi) * (np.tanh((omega**2*h/g)**(3/4)))**(2/3)
print(f"L = {L:.4f} m")

# 誤差百分比 (精確解 L = 92.373)
err = np.abs(L-92.373)/92.373 * 100
print(f"Relative error of L = {err:.2f}%")

L = 93.8785 m
Relative error of L = 1.63%


## 3. Guo (2002) 提出之近似解

\begin{equation}
k = \frac{\omega^2}{g}\left[1-e^{-(\omega\sqrt{h/g})^{5/2}} \right]^{-2/5}
\end{equation}

Source: Guo, J. (2002) Simple and explicit solution of wave dispersion equation, Coastal Engineering 45, 71–74.

### 測試例
T = 10 s, h = 10 m, 精確解為 L = 92.373 m。上式之近似解為

In [7]:
# Wave condition and parameters
g = 9.81
T = 10
h = 10
omega = 2*np.pi/T

# Approximation solution by Eckart (1951) 
k = omega**2/g * (1-np.exp(-(omega*np.sqrt(h/g))**(5/2)))**(-2/5)
L = 2*np.pi/k
print(f"L = {L:.4f} m")

# 誤差百分比 (精確解 L = 92.373)
err = np.abs(L-92.373)/92.373 * 100
print(f"Relative error of L = {err:.2f}%")

L = 93.0544 m
Relative error of L = 0.74%


# 使用 Newton Raphson 法求解頻散關係式 (最精確解)

source: https://github.com/pyoceans/python-oceans

使用方式 (詳見程式內註解)：
1. 先載入 `class Waves` (在同個 Jupyter Notebook 中載入一次即可)。
2. 呼叫 `Waves` 並輸入水深 `h` 與週期 `T`，即可輸出波長、波速、群速等值。
3. 亦可輸入水深 `h`、波長 `L` 以計算週期。

In [6]:
class Waves:
    r"""
    Solves the wave dispersion relationship via Newton-Raphson.

    .. math::
        \omega^2 = gk\tanh kh

    Parameters
    ----------
    h : array_like, str
        Water depth [m] or 'deep', 'shallow' as keywords
    T : array_like
        Wave period [s]
    L : array_like
        Wave length [m]
    thetao : array_like
        TODO
    Ho : array_like
        TODO

    Returns
    -------
    omega : array_like
            Wave frequency
    TODO: hoLo, hoL, Lo, L, k, T, Co, C, Cg, G, Ks, Kr, theta, H

    Notes
    -----
    Compare values with:
    http://www.coastal.udel.edu/faculty/rad/wavetheory.html

    Examples
    --------
    >>> from oceans.sw_extras.waves import Waves
    >>> wav = Waves(h=10, T=5, L=None)
    >>> print(f"ho/Lo = {wav.hoLo:.3f}")
    ho/Lo = 0.256
    >>> print(f"ho/L  = {wav.hoL:.3f}")
    ho/L  = 0.273
    >>> print(f"Lo    = {wav.Lo:.3f}")
    Lo    = 39.033
    >>> print(f"L     = {wav.L:.3f}")
    L     = 36.593
    >>> print(f"k     = {wav.k:.3f}")
    k     = 0.172
    >>> print(f"omega = {wav.omega:.3f}")
    omega = 1.257
    >>> print(f"T     = {wav.T:.3f}")
    T     = 5.000
    >>> print(f"C     = {wav.C:.3f}")
    C     = 7.319
    >>> print(f"Cg    = {wav.Cg:.3f}")
    Cg    = 4.471
    >>> print(f"G     = {wav.G:.3f}")
    G     = 0.222
    >>> wav = Waves(h=10, T=None, L=100)
    >>> print(f"ho/Lo = {wav.hoLo:.3f}")
    ho/Lo = 0.056
    >>> print(f"ho/L  = {wav.hoL:.3f}")
    ho/L  = 0.100
    >>> print(f"Lo    = {wav.Lo:.3f}")
    Lo    = 179.568
    >>> print(f"L     = {wav.L:.3f}")
    L     = 100.000
    >>> print(f"k     = {wav.k:.3f}")
    k     = 0.063
    >>> print(f"omega = {wav.omega:.3f}")
    omega = 0.586
    >>> print(f"T     = {wav.T:.3f}")
    T     = 10.724
    >>> print(f"C     = {wav.C:.3f}")
    C     = 9.325
    >>> print(f"Cg    = {wav.Cg:.3f}")
    Cg    = 8.291
    >>> print(f"G     = {wav.G:.3f}")
    G     = 0.778
    >>> print(f"Ks  = {wav.Ks:.3f}")
    Ks  = 1.005

    """

    def __init__(self, h, T=None, L=None, thetao=None, Ho=None, lat=None):
        self.T = np.asarray(T, dtype=np.float64)
        self.L = np.asarray(L, dtype=np.float64)
        self.Ho = np.asarray(Ho, dtype=np.float64)
        self.lat = np.asarray(lat, dtype=np.float64)
        self.thetao = np.asarray(thetao, dtype=np.float64)

        if isinstance(h, str):
            if L is not None:
                if h == "deep":
                    self.h = self.L / 2.0
                elif h == "shallow":
                    self.h = self.L * 0.05
        else:
            self.h = np.asarray(h, dtype=np.float64)

        g = 9.81  # Default gravity.

        if L is None:
            self.omega = 2 * np.pi / self.T
            self.Lo = (g * self.T**2) / 2 / np.pi
            # Returns wavenumber of the gravity wave dispersion relation using
            # newtons method. The initial guess is shallow water wavenumber.
            self.k = self.omega / np.sqrt(g)
            # TODO: May change to,
            # self.k = self.w ** 2 / (g * np.sqrt(self.w ** 2 * self.h / g))
            f = g * self.k * np.tanh(self.k * self.h) - self.omega**2

            while np.abs(f.max()) > 1e-10:
                dfdk = g * self.k * self.h * (
                    1 / (np.cosh(self.k * self.h))
                ) ** 2 + g * np.tanh(self.k * self.h)
                self.k = self.k - f / dfdk
                # FIXME:
                f = g * self.k * np.tanh(self.k * self.h) - self.omega**2

            self.L = 2 * np.pi / self.k
            if isinstance(h, str):
                if h == "deep":
                    self.h = self.L / 2.0
                elif h == "shallow":
                    self.h = self.L * 0.05
        else:
            self.Lo = self.L / np.tanh(2 * np.pi * self.h / self.L)
            self.k = 2 * np.pi / self.L
            self.T = np.sqrt(2 * np.pi * self.Lo / g)
            self.omega = 2 * np.pi / self.T

        self.hoL = self.h / self.L
        self.hoLo = self.h / self.Lo
        self.C = self.omega / self.k  # or L / T
        self.Co = self.Lo / self.T
        self.G = 2 * self.k * self.h / np.sinh(2 * self.k * self.h)
        self.n = (1 + self.G) / 2
        self.Cg = self.n * self.C
        self.Ks = np.sqrt(1 / (1 + self.G) / np.tanh(self.k * self.h))

        if thetao is None:
            self.theta = np.NaN
            self.Kr = np.NaN
        if thetao is not None:
            self.theta = np.rad2deg(
                np.asin(self.C / self.Co * np.sin(np.deg2rad(self.thetao))),
            )
            self.Kr = np.sqrt(
                np.cos(np.deg2rad(self.thetao)) / np.cos(np.deg2rad(self.theta)),
            )

        if Ho is None:
            self.H = np.NaN
        if Ho is not None:
            self.H = self.Ho * self.Ks * self.Kr

### 範例一
計算 h = 10 m, T = 10 s 之波長 L, 波速 C 與群速 Cg：

In [7]:
wav = Waves(h=10, T=10, L=None)  # 未知數需輸入 None
wav.L                            # wave.L 代表輸出波長

92.37387271170788

In [8]:
wav.C   # 波速

9.237387271170787

In [9]:
wav.Cg  # 群速

8.06993413970063

或是使用 `print` 指令做較詳盡的輸出：

In [10]:
wav = Waves(h=10, T=10, L=None)
print(f"L  = {wav.L:.3f} m")
print(f"C  = {wav.C:.3f} m/s")
print(f"Cg = {wav.Cg:.3f} m/s")

L  = 92.374 m
C  = 9.237 m/s
Cg = 8.070 m/s


### 範例二
計算 h = 10 m, L = 92.374 m 之週期 T:

In [11]:
wav = Waves(h=10, T=None, L=92.374)
print(f"T  = {wav.T:.3f} s")

T  = 10.000 s


### 範例三
計算深水處 h = 100 m, T = 10 s 之波長 L:

In [12]:
wav = Waves(h=100, T=10, L=None)
print(f"L = {wav.L:.3f} m")

L = 156.032 m


### 範例四
計算淺水處 h = 2.3 m, T = 10 s 之波長 L：

In [13]:
wav = Waves(h=2.3, T=10, L=None)
print(f"L = {wav.L:.3f} m")

L = 46.767 m
